In [17]:
from __future__ import division
from vincenty import vincenty
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import geoplotlib as gp
import scipy as sp
import gdal 
from gdalconst import * 
import datetime
import math
import operator
import re
import os
import os.path
import requests, time
import urllib, json
import csv
from sklearn import neighbors, datasets,linear_model,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import LeavePLabelOut
from sklearn.preprocessing import scale
from sklearn.metrics.pairwise import euclidean_distances
from pylab import *
from scipy import stats
from sklearn.neighbors import DistanceMetric
from sklearn import metrics
import datetime as dt 

#for parallel processing
from ipyparallel import Client

import netCDF4
from netCDF4 import Dataset

%matplotlib inline

### Data Processing 

#### Humidity Interpretation

In [24]:
year='2016'
hum_year=Dataset('data\\daily NCEP dataframes\\rhum.'+year+'.nc')
print "HUM Data Dimension: ",hum_year.dimensions.keys()

HUM Data Dimension:  [u'lon', u'lat', u'level', u'time', u'nbnds']


In [33]:
hum_year.variables['rhum']

<type 'netCDF4._netCDF4.Variable'>
int16 rhum(time, level, lat, lon)
    long_name: daily mean 6-hourly Relative Humidity on Pressure Levels
    valid_range: [-32765 -17766]
    unpacked_valid_range: [ -25.  125.]
    actual_range: [  -5.75  106.75]
    units: %
    add_offset: 302.65
    scale_factor: 0.01
    missing_value: 32766
    _FillValue: -32767
    precision: 2
    least_significant_digit: 0
    GRIB_id: 52
    GRIB_name: RH
    var_desc: Relative humidity
    dataset: NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Daily Averages
    level_desc: Pressure Levels
    statistic: Mean
    parent_stat: Individual Obs
    standard_name: relative_humidity
unlimited dimensions: time
current shape = (366, 17, 73, 144)
filling off

In [38]:
hum_lats = hum_year.variables['lat'][:]  
hum_lons = hum_year.variables['lon'][:]
rhum_var = hum_year.variables['rhum'][:]
time_var = hum_year.variables['time']

dtime = netCDF4.num2date(time_var[:],time_var.units)

first = netCDF4.num2date(time_var[0],time_var.units)
last = netCDF4.num2date(time_var[-1],time_var.units)
first_date = first.strftime('%Y%m%d')
last_date = last.strftime('%Y%m%d')

#### Cloud Cover Interpolation

In [41]:
cc_year=Dataset('data\\daily NCEP dataframes\\tcdc.eatm.gauss.'+year+'.nc')
print "Cloud Cover Data Dimension: ",cc_year.dimensions.keys()

Cloud Cover Data Dimension:  [u'lon', u'lat', u'nbnds', u'time']


In [40]:
cc_year.variables['tcdc']

<type 'netCDF4._netCDF4.Variable'>
int16 tcdc(time, lat, lon)
    long_name: daily mean 6-Hourly Mean of Total Cloud Cover at Entire Atmosphere Considered as a Single Layer
    valid_range: [-32765 -31765]
    unpacked_valid_range: [   0.  100.]
    actual_range: [   0.  100.]
    units: %
    add_offset: 3276.5
    scale_factor: 0.1
    missing_value: 32766
    _FillValue: -32767
    precision: 1
    least_significant_digit: 0
    GRIB_id: 71
    GRIB_name: TCDC
    var_desc: Total cloud cover
    dataset: NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Daily Averages
    level_desc: Entire Atmosphere Considered As a Single Layer
    statistic: Mean
    parent_stat: Individual Obs
    standard_name: cloud_area_fraction
    cell_methods: time: mean
unlimited dimensions: time
current shape = (366, 94, 192)
filling off

In [ ]:
cc_lats = cc_year.variables['lat'][:]  
cc_lons = cc_year.variables['lon'][:]
cc_var = cc_year.variables['rhum'][:]
time_var = cc_year.variables['time']

dtime = netCDF4.num2date(time_var[:],time_var.units)

first = netCDF4.num2date(time_var[0],time_var.units)
last = netCDF4.num2date(time_var[-1],time_var.units)
first_date = first.strftime('%Y%m%d')
last_date = last.strftime('%Y%m%d')

#### Wind Interpolation

In [45]:
vw_year=Dataset('data\\daily NCEP dataframes\\vwnd.10m.gauss.'+year+'.nc')
uw_year=Dataset('data\\daily NCEP dataframes\\uwnd.10m.gauss.'+year+'.nc')
print "V Wind Data Dimension: ",vw_year.dimensions.keys()
print "U Wind Data Dimension: ",uw_year.dimensions.keys()

V Wind Data Dimension:  [u'lon', u'lat', u'level', u'time', u'nbnds']
U Wind Data Dimension:  [u'lon', u'lat', u'level', u'time', u'nbnds']


In [48]:
print vw_year.variables['vwnd']
print "-------------------------------------------------------"
print uw_year.variables['uwnd']

<type 'netCDF4._netCDF4.Variable'>
int16 vwnd(time, level, lat, lon)
    long_name: daily mean 6-Hourly Forecast of V-wind at 10 m
    valid_range: [-32765  -8765]
    unpacked_valid_range: [-120.  120.]
    actual_range: [-32.13999939  29.38999939]
    units: m/s
    add_offset: 207.65
    scale_factor: 0.01
    missing_value: 32766
    _FillValue: -32767
    precision: 2
    least_significant_digit: 1
    GRIB_id: 34
    GRIB_name: V GRD
    var_desc: v-wind
    dataset: NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Daily Averages
    level_desc: 10 m
    statistic: Mean
    parent_stat: Individual Obs
    standard_name: northward_wind
unlimited dimensions: time
current shape = (366, 1, 94, 192)
filling off

-------------------------------------------------------
<type 'netCDF4._netCDF4.Variable'>
int16 uwnd(time, level, lat, lon)
    long_name: daily mean 6-Hourly Forecast of U-wind at 10 m
    valid_range: [-32765  -8765]
    unpacked_valid_range: [-120.  120.]
    actual_range: [-34.